In [1]:
from dotenv import load_dotenv

load_dotenv()

from rich.console import Console
console = Console(width=96)

## Creating subagents

In [2]:
from langchain.tools import tool

@tool
def square_root(x: float) -> float:
    """Calculate the square root of a number"""
    return x ** 0.5

@tool
def square(x: float) -> float:
    """Calculate the square of a number"""
    return x ** 2

In [3]:
from langchain.agents import create_agent

# create subagents

subagent_1 = create_agent(
    model='gpt-5-nano',
    tools=[square_root]
)

subagent_2 = create_agent(
    model='gpt-5-nano',
    tools=[square]
)

## Calling subagents

In [4]:
from langchain.messages import HumanMessage

@tool
def call_subagent_1(x: float) -> float:
    """Call subagent 1 in order to calculate the square root of a number"""
    response = subagent_1.invoke({"messages": [HumanMessage(content=f"Calculate the square root of {x}")]})
    return response["messages"][-1].content

@tool
def call_subagent_2(x: float) -> float:
    """Call subagent 2 in order to calculate the square of a number"""
    response = subagent_2.invoke({"messages": [HumanMessage(content=f"Calculate the square of {x}")]})
    return response["messages"][-1].content

## Creating the main agent

main_agent = create_agent(
    model='gpt-5-nano',
    tools=[call_subagent_1, call_subagent_2],
    system_prompt="You are a helpful assistant who can call subagents to calculate the square root or square of a number.")

## Test

In [5]:
question = "What is the square root of 456?"

response = main_agent.invoke({"messages": [HumanMessage(content=question)]})

In [6]:
from pprint import pprint

console.print(response)

{
    'messages': [
        HumanMessage(
            content='What is the square root of 456?',
            additional_kwargs={},
            response_metadata={},
            id='1b9675cf-9150-4449-9419-cf6bb2f0dd52'
        ),
        AIMessage(
            content='',
            additional_kwargs={'refusal': None},
            response_metadata={
                'token_usage': {
                    'completion_tokens': 922,
                    'prompt_tokens': 202,
                    'total_tokens': 1124,
                    'completion_tokens_details': {
                        'accepted_prediction_tokens': 0,
                        'audio_tokens': 0,
                        'reasoning_tokens': 896,
                        'rejected_prediction_tokens': 0
                    },
                    'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}
                },
                'model_provider': 'openai',
                'model_name': 'gpt-5-nano-2025-08-07',
                'system_fingerprint': None,
                'id': 'chatcmpl-CyPYjG2htrEn5CKcftTTq6JeT0NO1',
                'service_tier': 'default',
                'finish_reason': 'tool_calls',
                'logprobs': None
            },
            id='lc_run--019bc3a5-8b35-7341-a675-f4039d89ae9e-0',
            tool_calls=[
                {
                    'name': 'call_subagent_1',
                    'args': {'x': 456},
                    'id': 'call_78dqMtjpZKbjZXdJwCpwdOmd',
                    'type': 'tool_call'
                }
            ],
            usage_metadata={
                'input_tokens': 202,
                'output_tokens': 922,
                'total_tokens': 1124,
                'input_token_details': {'audio': 0, 'cache_read': 0},
                'output_token_details': {'audio': 0, 'reasoning': 896}
            }
        ),
        ToolMessage(
            content='The square root of 456.0 is approximately 21.3541565 (about 21.3542).',
            name='call_subagent_1',
            id='8266bc3f-e42f-4396-b742-04b893b3f3c5',
            tool_call_id='call_78dqMtjpZKbjZXdJwCpwdOmd'
        ),
        AIMessage(
            content='√456 ≈ 21.3542 (approximately; more precisely about 21.3541565).',
            additional_kwargs={'refusal': None},
            response_metadata={
                'token_usage': {
                    'completion_tokens': 351,
                    'prompt_tokens': 260,
                    'total_tokens': 611,
                    'completion_tokens_details': {
                        'accepted_prediction_tokens': 0,
                        'audio_tokens': 0,
                        'reasoning_tokens': 320,
                        'rejected_prediction_tokens': 0
                    },
                    'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}
                },
                'model_provider': 'openai',
                'model_name': 'gpt-5-nano-2025-08-07',
                'system_fingerprint': None,
                'id': 'chatcmpl-CyPZ2Ja3fXgjGHmhzZcI0vAk3IqnC',
                'service_tier': 'default',
                'finish_reason': 'stop',
                'logprobs': None
            },
            id='lc_run--019bc3a5-d6a9-7d62-9b05-40b68bc3dfc9-0',
            usage_metadata={
                'input_tokens': 260,
                'output_tokens': 351,
                'total_tokens': 611,
                'input_token_details': {'audio': 0, 'cache_read': 0},
                'output_token_details': {'audio': 0, 'reasoning': 320}
            }
        )
    ]
}